In [1]:
# API 키를 환경변수로 관리하기 위한 설정 파일
from dotenv import load_dotenv

# API 키 정보 로드
load_dotenv()

True

In [2]:
# LangSmith 추적을 설정합니다. https://smith.langchain.com
# !pip install langchain-teddynote
from langchain_teddynote import logging

# 프로젝트 이름을 입력합니다.
logging.langsmith("self_query")

LangSmith 추적을 시작합니다.
[프로젝트명]
sekf_query


In [3]:
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document
from langchain_openai import OpenAIEmbeddings

In [4]:
from langchain_community.document_loaders.csv_loader import CSVLoader

# CSV 로더 생성
loader = CSVLoader(file_path="/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files_yj/MD_menu_cleaned.csv")

# 데이터 로드
docs = loader.load()

In [5]:
docs

[Document(metadata={'source': '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files_yj/MD_menu_cleaned.csv', 'row': 0}, page_content='menuname: 맥크리스피 스리라차 마요 버거\ncategory: 버거\nnewmenu: True\ndescription: 빠삭한  통닭다리살 케이준 패티에 스리라차 마요 소스를 더해 매콤 고소한 맛\ningredients: 난류,우유,대두,밀,토마토,닭고기,쇠고기\ngrams: 289.0\nkcals: 663.0\nburgerprice: 8600\nsetmenu: True\nsetprice: 10500.0'),
 Document(metadata={'source': '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files_yj/MD_menu_cleaned.csv', 'row': 1}, page_content='menuname: 베토디 스리라차 마요 버거\ncategory: 버거\nnewmenu: True\ndescription: 베이컨 토마토 디럭스에 스리라차 마요 소스를 더해 색다른 매콤함\ningredients: 난류,우유,대두,밀,돼지고기,토마토,쇠고기\ngrams: 251.0\nkcals: 621.0\nburgerprice: 7600\nsetmenu: True\nsetprice: 9500.0'),
 Document(metadata={'source': '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files_yj/MD_menu_cleaned.csv', 'row': 2}, page_content='menuname: 맥스파이시 상하이 버거\ncategory: 버거\nnewmenu: False\ndescription: 쌀가루가 더해져 더 바삭해진  닭가슴살 패티에 아삭아삭한 양상추와 신선한 토마토까지 더 바삭하고 맛있어진 N

In [6]:
# 벡터 저장소 생성
vectorstore = Chroma.from_documents(
    docs, OpenAIEmbeddings(model="text-embedding-3-large")
)

In [7]:
from langchain.chains.query_constructor.base import AttributeInfo


# 메타데이터 필드 정보 생성
metadata_field_info = [
    {
        'name': 'menuname',
        'description': "Name of the menu item. Valid values are ['1955 버거', '더블 불고기 버거', '더블 치즈버거', '더블 쿼터파운더 치즈 버거', '맥스파이시 상하이 버거', '맥치킨 모짜렐라 버거', '맥치킨 버거', '맥크리스피 디럭스 버거', '맥크리스피 스리라차 마요 버거', '맥크리스피 클래식 버거', '베이컨 토마토 디럭스 버거', '베토디 스리라차 마요 버거', '불고기 버거', '빅맥 버거', '슈비 버거', '슈슈 버거', '에그 불고기 버거', '치즈버거', '쿼터파운더 치즈 버거', '탄산 라지', '탄산 미디움', '토마토 치즈 비프 버거', '트리플 치즈버거', '햄버거', '후렌치후라이 라지', '후렌치후라이 미디움', '후렌치후라이 스몰']",
        'type': 'string'
    },
    {
        'name': 'category',
        'description': "Category of the menu item. Valid values are ['버거 ', '사이드', '음료 ']",
        'type': 'string'
    },
    {
        'name': 'newmenu',
    'description': 'Indicates if the menu item is new',
    'type': 'boolean'
    },
    {
        'name': 'description',
        'description': 'Description of the menu item',
        'type': 'string'
    },
    {
        'name': 'ingredients',
        'description': "List of ingredients in the menu item. Valid values are ['굴', '난류', '닭고기', '대두', '돼지고기', '밀', '새우', '쇠고기', '우유', '조개', '토마토']",
        'type': 'string'
    },
    {
        'name': 'grams',
        'description': 'Weight of the menu item in grams',
        'type': 'float'
    }
    ,
    {
        'name': 'kcals',
        'description': 'Calories of the menu item',
        'type': 'float'
    },
    {
        'name': 'burgerprice',
        'description': 'Price of the burger item',
        'type': 'integer'
    },
    {
        'name': 'setmenu',
        'description': 'Indicates if the menu item is part of a set menu',
        'type': 'boolean'
    },
    {
        'name': 'setprice',
        'description': 'Price of the menu item as part of a set menu',
        'type': 'float'
    }
]

In [8]:
from langchain.retrievers.self_query.base import SelfQueryRetriever
from langchain_openai import ChatOpenAI

# LLM 정의
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# SelfQueryRetriever 생성
retriever = SelfQueryRetriever.from_llm(
    llm=llm,
    vectorstore=vectorstore,
    document_contents="Brief summary of a cosmetic product",
    metadata_field_info=metadata_field_info,
)


In [9]:
retriever.invoke("치킨 버거는 무슨무슨 종류가 있나요?")

[Document(metadata={'row': 11, 'source': '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files_yj/MD_menu_cleaned.csv'}, page_content='menuname: 맥치킨 버거\ncategory: 버거\nnewmenu: False\ndescription: 바삭한 치킨 패티 고소한 화이트 마요 소스와 아삭한 양상추가 함께 더 업그레이드된 맛으로 돌아온 맥치킨\ningredients: 난류,대두,밀,닭고기\ngrams: 206.0\nkcals: 523.0\nburgerprice: 4300\nsetmenu: True\nsetprice: 6700.0'),
 Document(metadata={'row': 13, 'source': '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files_yj/MD_menu_cleaned.csv'}, page_content='menuname: 에그 불고기 버거\ncategory: 버거\nnewmenu: False\ndescription: 불고기 버거에 국내산 등급 계란을 더해 맛의 정점을 찍다\ningredients: 난류,대두,밀,돼지고기,조개,굴\ngrams: 222.0\nkcals: 491.0\nburgerprice: 5100\nsetmenu: True\nsetprice: 7300.0'),
 Document(metadata={'row': 8, 'source': '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files_yj/MD_menu_cleaned.csv'}, page_content='menuname: 맥크리스피 클래식 버거\ncategory: 버거\nnewmenu: True\ndescription: 통닭다리살 겉바속촉 케이준 치킨 패티 치킨 버거 본연의 맛에 충실한 클래식 버거\ningredients: 난류,우유,대두,밀,토마토,닭고기,쇠

In [10]:
retriever.invoke("제일 열량이 낮은 메뉴를 추천해 줘")

[Document(metadata={'row': 23, 'source': '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files_yj/MD_menu_cleaned.csv'}, page_content='menuname: 탄산 라지\ncategory: 음료\nnewmenu: False\ndescription: 갈증해소 뿐만이 아니라 기분까지 상쾌하게 코카콜라\ningredients: \ngrams: \nkcals: \nburgerprice: 3100\nsetmenu: \nsetprice: '),
 Document(metadata={'row': 15, 'source': '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files_yj/MD_menu_cleaned.csv'}, page_content='menuname: 슈슈 버거\ncategory: 버거\nnewmenu: False\ndescription: 탱글한 통새우살 가득 슈슈 버거\ningredients: 난류,우유,대두,밀,토마토,새우,쇠고기,굴\ngrams: 224.0\nkcals: 409.0\nburgerprice: 5500\nsetmenu: True\nsetprice: 7300.0'),
 Document(metadata={'row': 22, 'source': '/home/yoojin/ML/aiffel/HackaThon/modu_hackaton/LLM/files_yj/MD_menu_cleaned.csv'}, page_content='menuname: 탄산 미디움\ncategory: 음료\nnewmenu: False\ndescription: 갈증해소 뿐만이 아니라 기분까지 상쾌하게 코카콜라\ningredients: \ngrams: \nkcals: \nburgerprice: 2600\nsetmenu: \nsetprice: '),
 Document(metadata={'row': 12, 'source': '/home/yoo

In [11]:
from langchain.chains.query_constructor.base import (
    StructuredQueryOutputParser,
    get_query_constructor_prompt,
)

# 문서 내용 설명과 메타데이터 필드 정보를 사용하여 쿼리 생성기 프롬프트를 가져옵니다.
prompt = get_query_constructor_prompt(
    "Brief summary of a McDonald's menu",  # 문서 내용 설명
    metadata_field_info,  # 메타데이터 필드 정보
)

# StructuredQueryOutputParser 를 생성
output_parser = StructuredQueryOutputParser.from_components()

# query_constructor chain 을 생성
query_constructor = prompt | llm | output_parser


In [12]:
query_output = query_constructor.invoke(
    {
        # 쿼리 생성기를 호출하여 주어진 질문에 대한 쿼리를 생성합니다.
        "query": "닭고기가 들어가는 메뉴 중에 가장 열량이 낮는 메뉴를 추천해주세요"
    }
)

In [14]:
query_output

StructuredQuery(query='닭고기', filter=None, limit=None)

In [16]:
from langchain_huggingface.embeddings import HuggingFaceEmbeddings

model_name = "intfloat/multilingual-e5-large-instruct"

hf_embeddings = HuggingFaceEmbeddings(
    model_name=model_name,
    model_kwargs={"device": "cuda"},  # cuda, cpu
    encode_kwargs={"normalize_embeddings": True},
)

/home/yoojin/anaconda3/envs/RAG/lib/python3.10/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm, trange


In [17]:
embedded_documents1 = hf_embeddings.embed_documents(texts)

AttributeError: 'Document' object has no attribute 'replace'